In [1]:
import gspread
from gspread_formatting import cellFormat, Color, textFormat, format_cell_range, batch_updater, ConditionalFormatRule,  get_conditional_format_rules, GridRange, BooleanRule, BooleanCondition, CellFormat, ConditionValue
import os
import numpy as np
import pandas as pd
import re
import sys
from minio import Minio, S3Error

path = os.path.normpath(os.path.join(os.getcwd(), "../airflow/dags"))
sys.path.append(path)
from gamedata.helpers import get_s3_file, put_s3_file, s3_path_exists

PATH_WIKIPEDIA_RESULT = f'wikipedia_result.csv'
PATH_METACRITIC_RESULT = f'metacritic_result.csv'

MANUAL_MAP = [
    ['Action-adv', 'Action-Adventure'],
    ['Action-adventure game', 'Action-Adventure'],
    ['Action-platform', 'Action-Platformer'],
    ['Action-role playing', 'Action Role-Playing'],
    ['Acton role-playing', 'Action Role-Playing'],
    ['Adventure video game', 'Adventure'],
    ['art game', 'Art'],
    ['Avtion-platform', 'Platform'],
    ['Construction and management', 'Construction And Management Simulation'],
    ['dungeon crawl', 'Dungeon Crawler'],
    ['Dungeon crawl', 'Dungeon Crawler'],
    ['farm simulation', 'Farming Simulation'],
    ['Farm simulation', 'Farming Simulation'],
    ['fighter', 'Fighting'],
    ['First- and third-person shooter', 'First-and-third-person Shooter'],
    ['FMV', 'Full Motion Video'],
    ['Hack and Slash', 'Hack & Slash'],
    ['Hack and slash', 'Hack & Slash'],
    ['hack and slash', 'Hack & Slash'],
    ['hack-and-slash', 'Hack & Slash'],
    ['Life', 'Life Simulation'],
    ['management', 'Management Simulation'],
    ['Massively multiplayer online', 'MMO'],
    ['Massively multiplayer online role-playing', 'MMORPG'],
    ['MMORPG', 'MMORPG'],
    ['Multidirectional shooter', 'Multi-Directional Shooter'],
    ['Physics', 'Physics Based'],
    ['Point and click adventure', 'Point & Click Adventure'],
    ['Point-and-click', 'Point & Click Adventure'],
    ['Point-and-click adventure', 'Point & Click Adventure'],
    ['Point-And-Click Adventure', 'Point & Click Adventure'],
    ['Puzzle platformer', 'Puzzle-Platformer'],
    ['Puzzle Platformer', 'Puzzle-Platformer'],
    ['puzzle-platform', 'Puzzle-Platformer'],
    ['rhythm-based', 'Rhythm'],
    ['rogue-like', 'Roguelike'],
    ['Role Playing', 'Role-Playing'],
    ['Rougelike', 'Roguelike'],
    ['rougelike', 'Roguelike'],
    ['run and gun', 'Run \'N Gun'],
    ['Run and gun', 'Run \'N Gun'],
    ['side-scrolling', 'Side-Scroller'],
    ['side-scrolling', 'Side-Scroller'],
    ['sidescroller', 'Side-Scroller'],
    ['simulator', 'Simulation'],
    ['Soulslike', 'Souls-Like'],
    ['soulslike', 'Souls-Like'],
    ['Survival game', 'Survival'],
]


In [2]:
# Load data
df_wikipedia = get_s3_file(PATH_WIKIPEDIA_RESULT)
df_metacritic = get_s3_file(PATH_METACRITIC_RESULT, lineterminator='\n')

df_wikipedia = df_wikipedia.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
df_metacritic = df_metacritic.drop(['Unnamed: 0'], axis=1)

In [21]:
# Merge dataframes
df_core = df_wikipedia.merge(df_metacritic, how='left', left_on='Hash', right_on='Wikipedia Hash', suffixes=('_wp', '_mc'))

# Drop unnecessary columns
df_core = df_core.drop([
    'Title_mc',
    'Response Code',
    'Platform_mc',
    'Wikipedia Hash',
    'Publisher_mc',
    'Release',
    'Developer_mc',
    'Genres',
    'userscore Count'
], axis=1)

# Rename columns
df_core = df_core.rename(columns={
    'Title_wp': 'Title',
    'Developer_wp': 'Developer',
    'Publisher_wp': 'Publisher',
    'Unreleased NA': 'NA Release',
    'TBA NA': 'NA TBA',
    'Release Date NA': 'NA Release Date',
    'Unreleased JP': 'JP Release',
    'TBA JP': 'JP TBA',
    'Release Date JP': 'JP Release Date',
    'Unreleased PAL': 'PAL Release',
    'TBA PAL': 'PAL TBA',
    'Release Date PAL': 'PAL Release Date',
    'Platform_wp': 'Platform',
    'Hash_wp': 'Hash Wikipedia',
    'URL': 'Metacritic Link',
    'Hash_mc': 'Hash Metacritic',
    'Image': 'Metacritic Image',
})

# Invert release columns
df_core['NA Release'] = df_core['NA Release'].map(lambda c: not c)
df_core['JP Release'] = df_core['JP Release'].map(lambda c: not c)
df_core['PAL Release'] = df_core['PAL Release'].map(lambda c: not c)

# Fix Userscore 'tbd'
df_core['Userscore'] = df_core['Userscore'].map(lambda c: np.NAN if c == 'tbd' else c)
df_core['Userscore'] = df_core['Userscore'].astype('float')

# Fix Userscore Count
df_core['Userscore Count'] = df_core['Userscore Count'].map(lambda c: re.sub(r"[^0-9.]", '', str(c)))
df_core['Userscore Count'] = df_core['Userscore Count'].map(lambda c: 0 if c == '' else int(c) )

# Reorder columns
df_core = df_core.reindex(columns=[
    'Title',
    'Platform',
    'Developer',
    'Publisher',
    'Genre',
    'Rating',
    'Metascore',
    'Metascore Count',
    'Userscore',
    'Userscore Count',
    'NA Release',
    'NA TBA',
    'NA Release Date',
    'JP Release',
    'JP TBA',
    'JP Release Date',
    'PAL Release',
    'PAL TBA',
    'PAL Release Date',
    'Summary',
    'Players',
    'Crossbuy',
    'Crossplay',
    '3DTV',
    'PS Camera',
    'PS4 Pro Enhanced',
    'Play Link',
    'PSVR',
    'PSVR2',
    'Metacritic Image',
    'Wikipedia Link',
    'Metacritic Link',
    'Hash Wikipedia',
    'Hash Metacritic',
])

df_core


,Title,Platform,Developer,Publisher,Genre,Rating,Metascore,Metascore Count,Userscore,Userscore Count,...,PS Camera,PS4 Pro Enhanced,Play Link,PSVR,PSVR2,Metacritic Image,Wikipedia Link,Metacritic Link,Hash Wikipedia,Hash Metacritic
0,#Funtime,PS4,OneGuyGames,Quantum Astrophysicists Guild,"(""Shoot 'em up"",)",E,NaN,1.0,NaN,0,...,False,False,False,False,False,https://static.metacritic.com/images/products/...,NaN,https://www.metacritic.com/game/playstation-4/...,232d1f9a03a7935732c2caaa600c8430bc1ab62d7e5337...,389deb5a826f15bf6bee3d99f52ad288980b6f9d7c2e34...
1,#killallzombies,PS4,Beatshapers,Beatshapers,"('Shooter',)",T,63.0,6.0,6.6,19,...,False,False,False,False,False,https://static.metacritic.com/images/products/...,NaN,https://www.metacritic.com/game/playstation-4/...,2adf2c0fb216f44cb64bcc930c686814658fbaa996c3b8...,2adf2c0fb216f44cb64bcc930c686814658fbaa996c3b8...
2,.hack//G.U. Last Recode,PS4,CyberConnect2,Bandai Namco Entertainment,"('Action role-playing',)",T,76.0,21.0,8.1,72,...,False,True,False,False,False,https://static.metacritic.com/images/products/...,https://en.wikipedia.org/wiki/.hack//G.U._Last...,https://www.metacritic.com/game/playstation-4/...,eedf0475a44b790fe814a9a9432a3f9b7b91c2ab0ebffb...,ef3836ea960d6640b40155aa0f3fd45b48d228ec7af77a...
3,10 Second Ninja X,PS4,Four Circle Interactive,Curve Digital,"('Platform',)",T,76.0,16.0,5.5,23,...,False,False,False,False,False,https://static.metacritic.com/images/products/...,https://en.wikipedia.org/wiki/10_Second_Ninja_X,https://www.metacritic.com/game/playstation-4/...,8cd8b97deaa4f4e8ebe3ecf358c0c3eb33b8fe06704a16...,5ac8185ec846c8a56b3ae6072046e27afe3a23947d4207...
4,1001 Spikes,PS4,8bits Fanatics,Nicalis,"('Platform',)",T,78.0,8.0,6.5,37,...,False,False,False,False,False,https://static.metacritic.com/images/products/...,https://en.wikipedia.org/wiki/1001_Spikes,https://www.metacritic.com/game/playstation-4/...,73a95f52ae8c4af15d5c30604ff4d6f8920a1715d074ba...,fddae46320f1b4093fc340aa53e53ef4cd77bd72285f83...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3877,eFootball PES 2021 Season Update,PS4,PES Productions,Konami,"('Sports',)",NaN,NaN,NaN,NaN,0,...,False,False,False,False,False,NaN,https://en.wikipedia.org/wiki/EFootball_PES_20...,https://www.metacritic.com/game/playstation-4/...,b9685fe65e4edbabfbcc80384342d007cd7ea371998add...,NaN
3878,eFootball Pro Evolution Soccer 2020,PS4,PES Productions,Konami,"('Sports',)",NaN,NaN,NaN,NaN,0,...,False,False,False,False,False,NaN,https://en.wikipedia.org/wiki/EFootball_Pro_Ev...,https://www.metacritic.com/game/playstation-4/...,38ddbfefeeb077921a325cfd359ff28ba335276e51ae5f...,NaN
3879,iO,PS4,Gamious,Gamious,"('Platform, puzzle',)",E,68.0,7.0,NaN,0,...,False,False,False,False,False,https://static.metacritic.com/images/products/...,NaN,https://www.metacritic.com/game/playstation-4/io,28f7f508716f951474ce22e62990b6a563cb2bba3b9e2f...,28f7f508716f951474ce22e62990b6a563cb2bba3b9e2f...
3880,unWorded,PS4,Bento Studio,Bento Studio,"('Puzzle',)",NaN,NaN,NaN,NaN,0,...,False,False,False,False,False,NaN,NaN,https://www.metacritic.com/game/playstation-4/...,78578a4eb45e968d84f21da3a6f1b46e5a2597d33158c2...,NaN


In [23]:
# Replace NaN with '' for serialization
df_upload = df_core.fillna('')
# df_upload = df_upload.map('')


# Authenticate
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
gc = gspread.service_account('../airflow/psgamedata-key.json', scopes=scope)

# Open and clear the worksheet
sh = gc.open("Playstation Game Data")
wks = sh.worksheet('Data')
wks.clear()

# Upload
wks.update([df_upload.columns.values.tolist()] + df_upload.values.tolist())


/Users/crempp/projects/data-experiments/venv/lib/python3.9/site-packages/gspread/worksheet.py:1046: UserWarning: [Deprecated][in version 6.0.0]: method signature will change to: 'Worksheet.update(value = [[]], range_name=)' arguments 'range_name' and 'values' will swap, values will be mandatory of type: 'list(list(...))'
  warnings.warn(


{'spreadsheetId': '1x6c7ORLNIrt733poVKG1NVtS9QCqkDvJ2KVkQJJ9LhE',
 'updatedRange': 'Data!A1:AH3883',
 'updatedRows': 3883,
 'updatedColumns': 34,
 'updatedCells': 132022}

In [28]:
# Format Worksheet
LAST_COLUMN = 'AH'
rules = get_conditional_format_rules(wks)

# Reset
rules.clear()
all_fmt = cellFormat(
    backgroundColor=Color.fromHex('#ffffff'),
    textFormat=textFormat(
        bold=False,
        foregroundColor=Color.fromHex('#000000'),
        fontFamily='Ariel',
        fontSize=9
    ),
    horizontalAlignment='LEFT'
)
format_cell_range(wks, f'A:{LAST_COLUMN}', all_fmt)

# Update some header names
wks.update('H1', 'Metascore\nCount')
wks.update('J1', 'Userscore\nCount')
wks.update('K1', 'NA\nRelease')
wks.update('M1', 'NA Release\nDate')
wks.update('N1', 'JP\nRelease')
wks.update('P1', 'JP Release\nDate')
wks.update('Q1', 'PAL\nRelease')
wks.update('S1', 'PAL Release\nDate')
wks.update('Z1', 'PS4 Pro\nEnhanced')


# Header style
header_fmt = cellFormat(
    backgroundColor=Color.fromHex('#d6d6d6'),
    textFormat=textFormat(bold=True, foregroundColor=Color.fromHex('#3f3f3f')),
    horizontalAlignment='CENTER'
)

# Conditional Formatting
rule_metascore_1 = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('G2:G', wks)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('NUMBER_BETWEEN', ['0', '65']),
        format=CellFormat(backgroundColor=Color.fromHex('#f4c7c3'))
    )
)
rule_metascore_2 = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('G2:G', wks)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('NUMBER_BETWEEN', ['65', '80']),
        format=CellFormat(backgroundColor=Color.fromHex('#fce8b2'))
    )
)
rule_metascore_3 = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('G2:G', wks)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('NUMBER_BETWEEN', ['80', '100']),
        format=CellFormat(backgroundColor=Color.fromHex('#b7e1cd'))
    )
)
rule_userscore_1 = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('I2:I', wks)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('NUMBER_BETWEEN', ['0', '6.5']),
        format=CellFormat(backgroundColor=Color.fromHex('#f4c7c3'))
    )
)
rule_userscore_2 = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('I2:I', wks)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('NUMBER_BETWEEN', ['6.5', '8']),
        format=CellFormat(backgroundColor=Color.fromHex('#fce8b2'))
    )
)
rule_userscore_3 = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('I2:I', wks)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('NUMBER_BETWEEN', ['8', '10']),
        format=CellFormat(backgroundColor=Color.fromHex('#b7e1cd'))
    )
)
rule_na_release_true = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('K2:L', wks)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_EQ', ['TRUE']),
        format=CellFormat(backgroundColor=Color.fromHex('#b7e1cd'))
    )
)
rule_na_release_false = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('K2:L', wks)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_EQ', ['FALSE']),
        format=CellFormat(backgroundColor=Color.fromHex('#f4c7c3'))
    )
)
rule_jp_release_true = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('N2:O', wks)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_EQ', ['TRUE']),
        format=CellFormat(backgroundColor=Color.fromHex('#b7e1cd'))
    )
)
rule_jp_release_false = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('N2:O', wks)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_EQ', ['FALSE']),
        format=CellFormat(backgroundColor=Color.fromHex('#f4c7c3'))
    )
)
rule_pal_release_true = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('Q2:R', wks)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_EQ', ['TRUE']),
        format=CellFormat(backgroundColor=Color.fromHex('#b7e1cd'))
    )
)
rule_pal_release_false = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('Q2:R', wks)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_EQ', ['FALSE']),
        format=CellFormat(backgroundColor=Color.fromHex('#f4c7c3'))
    )
)
rule_capabilities_true = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('V2:AC', wks)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_EQ', ['TRUE']),
        format=CellFormat(backgroundColor=Color.fromHex('#b7e1cd'))
    )
)
rule_capabilities_false = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('V2:AC', wks)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_EQ', ['FALSE']),
        format=CellFormat(backgroundColor=Color.fromHex('#f4c7c3'))
    )
)

rule_plat = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range(f'A2:{LAST_COLUMN}', wks)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('CUSTOM_FORMULA', ['=$B2="PS4"']),
        format=CellFormat(backgroundColor=Color.fromHex('#e5eef7'))
    )
)

rules.append(rule_metascore_1)
rules.append(rule_metascore_2)
rules.append(rule_metascore_3)
rules.append(rule_userscore_1)
rules.append(rule_userscore_2)
rules.append(rule_userscore_3)
rules.append(rule_na_release_true)
rules.append(rule_na_release_false)
rules.append(rule_jp_release_true)
rules.append(rule_jp_release_false)
rules.append(rule_pal_release_true)
rules.append(rule_pal_release_false)
rules.append(rule_capabilities_true)
rules.append(rule_capabilities_false)

rules.append(rule_plat)
rules.save()

with batch_updater(wks.spreadsheet) as batch:
    batch.format_cell_range(wks, f'A1:{LAST_COLUMN}1', header_fmt)
    batch.set_column_width(wks, 'A', 300)
    batch.set_column_width(wks, 'B', 60)
    batch.set_column_width(wks, 'C', 200)
    batch.set_column_width(wks, 'D', 200)
    batch.set_column_width(wks, 'E', 200)
    batch.set_column_width(wks, 'F', 60)
    batch.set_column_width(wks, 'G', 60)
    batch.set_column_width(wks, 'H', 60)
    batch.set_column_width(wks, 'I', 60)
    batch.set_column_width(wks, 'J', 60)
    batch.set_column_width(wks, 'K', 60)
    batch.set_column_width(wks, 'L', 60)
    batch.set_column_width(wks, 'M', 80)
    batch.set_column_width(wks, 'N', 60)
    batch.set_column_width(wks, 'O', 60)
    batch.set_column_width(wks, 'P', 80)
    batch.set_column_width(wks, 'Q', 60)
    batch.set_column_width(wks, 'R', 60)
    batch.set_column_width(wks, 'S', 80)
    batch.set_column_width(wks, 'V', 60)
    batch.set_column_width(wks, 'W', 60)
    batch.set_column_width(wks, 'X', 60)
    batch.set_column_width(wks, 'Y', 60)
    batch.set_column_width(wks, 'Z', 60)
    batch.set_column_width(wks, 'AA', 60)
    batch.set_column_width(wks, 'AB', 60)
    batch.set_column_width(wks, 'AC', 60)
    batch.set_frozen(wks, rows=1, cols=1)



/Users/crempp/projects/data-experiments/venv/lib/python3.9/site-packages/gspread/worksheet.py:1046: UserWarning: [Deprecated][in version 6.0.0]: method signature will change to: 'Worksheet.update(value = [[]], range_name=)' arguments 'range_name' and 'values' will swap, values will be mandatory of type: 'list(list(...))'
  warnings.warn(
